AkHQ: http://localhost:8082
Spark-Master http://localhost:8083
Spark-Worker-1 http://localhost:8084

https://github.com/aehrc/pathling/tree/issue/452/lib/python#python-api-for-pathling

# Try Connect To Test FHIR Kafka

In [2]:
!python --version

Python 3.10.5


In [3]:
import os
os.environ

environ{'SHELL': '/bin/bash',
        'HOSTNAME': '33fb302b7bf4',
        'LANGUAGE': 'en_US.UTF-8',
        'SPARK_OPTS': '--driver-java-options=-Xms1024M --driver-java-options=-Xmx4096M --driver-java-options=-Dlog4j.logLevel=info',
        'NB_UID': '1000',
        'PWD': '/home/jovyan',
        'HOME': '/home/jovyan',
        'LANG': 'en_US.UTF-8',
        'NB_GID': '100',
        'XDG_CACHE_HOME': '/home/jovyan/.cache/',
        'APACHE_SPARK_VERSION': '3.3.0',
        'PYTHONPATH': '/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip:/usr/local/spark/python:',
        'HADOOP_VERSION': '3',
        'SHLVL': '0',
        'CONDA_DIR': '/opt/conda',
        'SPARK_HOME': '/usr/local/spark',
        'PATHLING_VERSION': '5.4.0',
        'SPARK_CONF_DIR': '/usr/local/spark/conf',
        'NB_USER': 'jovyan',
        'LC_ALL': 'en_US.UTF-8',
        'PATH': '/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/local/spark/bin',
        'PYSPARK_PYTHONPATH_SET': 

In [4]:
# for debugging
#os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1 pyspark-shell"

In [5]:
appName = "Kafka, Spark and FHIR Data"
master = "spark://spark-master:7077"
#master = "local[*]"
kafka_topic = "fhir.post-gateway-kdb"

In [6]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pathling.etc import find_jar

#conf = SparkConf()
#conf.setMaster(master) \
#    .setAppName(appName) \
#    .set("spark.ui.port","0")

# get current jars
#spark_jars = conf.get("spark.jars")
# append pathling jar
#updated_spark_jars = spark_jars + ",file://" + find_jar()

#updated_spark_jars = ["/test/1.jar", "/test2.jar"]

spark = SparkSession.builder \
    .appName(appName) \
    .master(master) \
    .config("spark.ui.port","0") \
    .getOrCreate()

spark.sparkContext.addFile(find_jar())

In [7]:
spark.sparkContext.getConf().get("spark.jars")
#updated_spark_jars

'file:///home/jovyan/.ivy2/jars/org.apache.spark_spark-sql-kafka-0-10_2.12-3.3.0.jar,file:///home/jovyan/.ivy2/jars/au.csiro.pathling_library-api-5.4.0.jar,file:///home/jovyan/.ivy2/jars/org.apache.spark_spark-token-provider-kafka-0-10_2.12-3.3.0.jar,file:///home/jovyan/.ivy2/jars/org.apache.kafka_kafka-clients-2.8.1.jar,file:///home/jovyan/.ivy2/jars/org.apache.commons_commons-pool2-2.11.1.jar,file:///home/jovyan/.ivy2/jars/org.spark-project.spark_unused-1.0.0.jar,file:///home/jovyan/.ivy2/jars/org.apache.hadoop_hadoop-client-runtime-3.3.2.jar,file:///home/jovyan/.ivy2/jars/org.lz4_lz4-java-1.8.0.jar,file:///home/jovyan/.ivy2/jars/org.xerial.snappy_snappy-java-1.1.8.4.jar,file:///home/jovyan/.ivy2/jars/org.apache.hadoop_hadoop-client-api-3.3.2.jar,file:///home/jovyan/.ivy2/jars/au.csiro.pathling_encoders-5.4.0.jar,file:///home/jovyan/.ivy2/jars/au.csiro.pathling_terminology-5.4.0.jar,file:///home/jovyan/.ivy2/jars/ch.qos.logback_logback-classic-1.2.11.jar,file:///home/jovyan/.ivy2/jar

In [8]:
# https://spark.apache.org/docs/latest/structured-streaming-kafka-integration.html
# default for startingOffsets is "latest", but "earliest" allows rewind for missed alerts
df = spark \
  .readStream  \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka1:19092") \
  .option("subscribe", kafka_topic) \
  .option("startingOffsets", "earliest") \
  .load()

In [9]:
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [10]:
query = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
          .writeStream \
          .queryName("gettable") \
          .format("memory") \
          .start()

# close connection after 30 seconds
query.awaitTermination(30)

False

In [11]:
kafka_data = spark.sql("select * from gettable")
kafka_data.show()
type(kafka_data)

+-------------------+--------------------+
|                key|               value|
+-------------------+--------------------+
|    Patient/patient|{"resourceType": ...|
|Condition/condition|{"resourceType": ...|
|Condition/condition|{"resourceType": ...|
|Procedure/procedure|{"resourceType": ...|
|Encounter/encounter|{"resourceType": ...|
+-------------------+--------------------+



pyspark.sql.dataframe.DataFrame

In [12]:
pd_df = kafka_data.toPandas()
pd_df

,key,value
0,Patient/patient,"{""resourceType"": ""Bundle"", ""meta"": {""security""..."
1,Condition/condition,"{""resourceType"": ""Bundle"", ""meta"": {""security""..."
2,Condition/condition,"{""resourceType"": ""Bundle"", ""meta"": {""security""..."
3,Procedure/procedure,"{""resourceType"": ""Bundle"",""meta"": {""security"":..."
4,Encounter/encounter,"{""resourceType"": ""Bundle"", ""meta"": {""security""..."


# Bring Pathling into the game

In [13]:
import pathling
from pathling import PathlingContext

ptl = PathlingContext.create(spark = spark)

In [14]:
# readme-example
#n_bundles = spark.read.text('/home/jovyan/pathling/lib/python/examples/data/bundles/', wholetext=True)
#n_bundles
#type(n_bundles)

In [15]:
kafka_data.select("value")

DataFrame[value: string]

In [16]:
patients = ptl.encode_bundle(kafka_data.select("value"), 'Patient')
encounter = ptl.encode_bundle(kafka_data.select("value"), 'Encounter')
condition = ptl.encode_bundle(kafka_data.select("value"), 'Condition')

In [17]:
encounter.printSchema()

root
 |-- id: string (nullable = true)
 |-- id_versioned: string (nullable = true)
 |-- meta: struct (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- versionId: string (nullable = true)
 |    |-- versionId_versioned: string (nullable = true)
 |    |-- lastUpdated: timestamp (nullable = true)
 |    |-- source: string (nullable = true)
 |    |-- profile: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- security: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- system: string (nullable = true)
 |    |    |    |-- version: string (nullable = true)
 |    |    |    |-- code: string (nullable = true)
 |    |    |    |-- display: string (nullable = true)
 |    |    |    |-- userSelected: boolean (nullable = true)
 |    |-- tag: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable

In [18]:
encounter.select("id", "subject.reference", "serviceType.coding.code", "period.start", "period.end").toPandas()

,id,reference,code,start,end
0,encounter,Patient/patient,"[2200, 2200]",2021-11-09T05:41:53+00:00,None


In [19]:
condition.printSchema()

root
 |-- id: string (nullable = true)
 |-- id_versioned: string (nullable = true)
 |-- meta: struct (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- versionId: string (nullable = true)
 |    |-- versionId_versioned: string (nullable = true)
 |    |-- lastUpdated: timestamp (nullable = true)
 |    |-- source: string (nullable = true)
 |    |-- profile: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- security: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- system: string (nullable = true)
 |    |    |    |-- version: string (nullable = true)
 |    |    |    |-- code: string (nullable = true)
 |    |    |    |-- display: string (nullable = true)
 |    |    |    |-- userSelected: boolean (nullable = true)
 |    |-- tag: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable

In [20]:
condition.select("id", "encounter.reference", "code.coding.code").toPandas()

,id,reference,code
0,condition,Encounter/encounter,[H57.0]
1,condition,Encounter/encounter,[H57.0]


In [21]:
from pyspark.sql.functions import regexp_replace, col

pt1 = patients.select("id", "birthDate", "gender", "address.postalCode")

enc1 = encounter \
    .select(
        col("id").alias("encounter_id"),
        "subject.reference",
        col("serviceType.coding.code").alias("servicetype_code"),
        col("period.start").alias("period_start"),
        col("period.end").alias("period_end")
       ) \
    .withColumn("patient_id", regexp_replace("reference", "Patient/", ""))

cd1 = condition \
    .select(
        "id",
        "encounter.reference",
        col("code.coding.code").alias("condition_code")
       ) \
    .withColumn("cond_encounter_id", regexp_replace("reference", "Encounter/", ""))

In [22]:
join1 = enc1 \
    .join(pt1, enc1.patient_id == pt1.id)
join2 = join1 \
    .select("encounter_id", "servicetype_code", "period_start", "period_end", "patient_id", "birthDate", "gender", "postalCode") \
    .join(cd1, join1.encounter_id == cd1.cond_encounter_id) \
    .select("encounter_id", "servicetype_code", "period_start", "period_end", "patient_id", "birthDate", "gender", "postalCode", "condition_code")

In [23]:
join2.toPandas()


,encounter_id,servicetype_code,period_start,period_end,patient_id,birthDate,gender,postalCode,condition_code
0,encounter,"[2200, 2200]",2021-11-09T05:41:53+00:00,None,patient,1990-01-01,female,[12345],[H57.0]
1,encounter,"[2200, 2200]",2021-11-09T05:41:53+00:00,None,patient,1990-01-01,female,[12345],[H57.0]
